In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.__version__

'2.9.0+cu126'

In [7]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")
print(f"GPU 이름: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU만 사용 가능'}")

사용 중인 장치: cuda
GPU 이름: NVIDIA GeForce RTX 3060 Ti


In [12]:
pip install transformers

  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------  11.8/12.0 MB 61.7 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 53.6 MB/s  0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 20.3 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 51.3 MB/s  0:00:00
Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl (320 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)

   ----------------------------------------  0/12 [urllib3]
   ----------------------------------------  0/12 [urll

In [13]:
from transformers import BertTokenizer

# 1. 설정
cfg = {
    "vocab_size": 30522,
    "max_len": 64,
    "hidden_size": 256,
    "type_vocab_size": 2,
    "dropout": 0.1
}

# 2. 임베딩 정의
class BertEmbedding(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.emb_word = nn.Embedding(cfg["vocab_size"], cfg["hidden_size"])
        self.emb_pos  = nn.Embedding(cfg["max_len"], cfg["hidden_size"])
        self.emb_type = nn.Embedding(cfg["type_vocab_size"], cfg["hidden_size"])
        self.ln = nn.LayerNorm(cfg["hidden_size"])
        self.do = nn.Dropout(cfg["dropout"])

    def forward(self, input_ids, token_type_ids):
        B, T = input_ids.shape
        device = input_ids.device
        pos = torch.arange(T, device=device).unsqueeze(0).expand(B, T)
        w = self.emb_word(input_ids)
        p = self.emb_pos(pos)
        t = self.emb_type(token_type_ids)
        x = self.ln(w + p + t)
        x = self.do(x)
        return x

emb = BertEmbedding(cfg)

# 3. 인터넷 예시 문장
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
text = "[CLS] i like coffee [SEP] but my friend likes tea [SEP]"

tokens = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=16)
print({k: v.shape for k, v in tokens.items()})  # input_ids, token_type_ids, attention_mask

# 4. 임베딩 통과
with torch.no_grad():
    out = emb(tokens["input_ids"], tokens["token_type_ids"])
print("Output shape:", out.shape)  # (1, 16, 256)

# 5. 확인
print("첫 5개 토큰:", tokenizer.convert_ids_to_tokens(tokens["input_ids"][0][:5]))
print("임베딩 첫 토큰 벡터 norm:", out[0,0].norm().item())


c:\projects\ai-implementation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'input_ids': torch.Size([1, 16]), 'token_type_ids': torch.Size([1, 16]), 'attention_mask': torch.Size([1, 16])}
Output shape: torch.Size([1, 16, 256])
첫 5개 토큰: ['[CLS]', '[CLS]', 'i', 'like', 'coffee']
임베딩 첫 토큰 벡터 norm: 16.568382263183594


In [ ]:
from transformers import BertTokenizer

# 1. 설정
cfg = {
    "vocab"
}